<a href="https://colab.research.google.com/github/Henil21/University-Research/blob/main/ITM_Research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MRI Classification For chest and brain diseases :
```
This notebook contain The implementation of research paper to be published bold 

author : Henilsinh raj, palak bhatt , Bhuvan tinani
```